In [2]:
!pip install tensorflow==1.15.0

     |████████████████████████████████| 124.0 MB 105 kB/s eta 0:00:01   |██▏                             | 8.3 MB 189 kB/s eta 0:10:12     |██▎                             | 8.8 MB 77 kB/s eta 0:24:50     |████▋                           | 17.8 MB 105 kB/s eta 0:16:47     |███████████▍                    | 44.2 MB 313 kB/s eta 0:04:15     |████████████▎                   | 47.6 MB 238 kB/s eta 0:05:21     |█████████████                   | 50.6 MB 237 kB/s eta 0:05:10     |███████████████▏                | 58.6 MB 195 kB/s eta 0:05:34     |████████████████▌               | 64.0 MB 182 kB/s eta 0:05:30     |████████████████▌               | 64.0 MB 182 kB/s eta 0:05:30     |█████████████████▍              | 67.3 MB 201 kB/s eta 0:04:42     |██████████████████▍             | 71.2 MB 154 kB/s eta 0:05:43     |██████████████████▍             | 71.2 MB 154 kB/s eta 0:05:43     |████████████████████▋           | 79.7 MB 121 kB/s eta 0:06:06     |█████████████████████           | 81.2 MB 115 

In [4]:
!pip install keras

  Using cached Keras-2.3.1-py2.py3-none-any.whl (377 kB)
     |████████████████████████████████| 28.5 MB 224 kB/s eta 0:00:01    |██████████▌                     | 9.3 MB 84 kB/s eta 0:03:48     |██████████████████████████      | 23.2 MB 113 kB/s eta 0:00:47     |████████████████████████████▍   | 25.3 MB 74 kB/s eta 0:00:43     |█████████████████████████████   | 25.8 MB 73 kB/s eta 0:00:37
     |████████████████████████████████| 269 kB 123 kB/s eta 0:00:01
  Created wheel for pyyaml: filename=PyYAML-5.3.1-cp36-cp36m-macosx_10_9_x86_64.whl size=44625 sha256=c295f23d3efe2543d17e3d8100a626edb5d4f49ad3ba489f1890b94da6e66e1f
  Stored in directory: /Users/soothingni/Library/Caches/pip/wheels/e5/9d/ad/2ee53cf262cba1ffd8afe1487eef788ea3f260b7e6232a80fc
Successfully built pyyaml


In [6]:
!pip install matplotlib

     |████████████████████████████████| 12.4 MB 21.9 MB/s eta 0:00:01   |███▎                            | 1.3 MB 824 kB/s eta 0:00:14
     |████████████████████████████████| 60 kB 11.6 MB/s eta 0:00:01


In [8]:
!pip install opencv-python

     |████████████████████████████████| 49.1 MB 648 kB/s eta 0:00:01    |██▊                             | 4.2 MB 273 kB/s eta 0:02:45     |██████▍                         | 9.8 MB 355 kB/s eta 0:01:51     |████████████▍                   | 18.9 MB 420 kB/s eta 0:01:12     |███████████████████▋            | 30.1 MB 330 kB/s eta 0:00:58


In [1]:
import tensorflow as tf
#from keras import backend as K
#from keras.layers import Input, BatchNormalization, LSTM, Dense, concatenate, Conv2D, MaxPooling2D, Flatten
#from keras.optimizers import Adam, SGD
#from keras.models import Model, Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import plot_model

import numpy as np
import random
import matplotlib.pyplot as plt
import os
import cv2
import json
import pathlib

Using TensorFlow backend.


In [19]:
from model.model_fn import model_fn
from model.utils import Params

In [20]:
model_dir = 'experiments/batch_hard'
data_dir = './img'

# Load the parameters from json file
json_path = os.path.join(model_dir, 'params.json')
params = Params(json_path)

In [21]:
data_root = pathlib.Path(data_dir)
all_image_paths = list(data_root.glob('**/*.jpg'))                                     #이미지 수준 파일 경로
all_directories = {'/'.join(str(i).split("/")[:-1]) for i in all_image_paths}    #폴더 수준 파일 경로

labels = list(i.split("/")[-1] for i in all_directories)

In [22]:
IMG_WIDTH, IMG_HEIGHT = 150, 150

In [27]:
def get_label_index(s):
        return labels.index(s.numpy().decode("utf-8").split("/")[-2])
    
def preprocess_image(image):
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.convert_image_dtype(image, tf.float32)
        image = tf.math.divide(image, 255.0)     
        return tf.image.resize(image, [IMG_WIDTH, IMG_HEIGHT])
    
def load_and_preprocess_image(path):
        image = tf.read_file(path)
        return tf.py_function(preprocess_image, [image], tf.float32), tf.py_function(get_label_index, [path], tf.int64)
    
def train_input_fn(data_dir, params):
    datasets = [tf.data.Dataset.list_files("{}/*.jpg".format(image_dir), shuffle=False) for image_dir in all_directories]
    # Create the list of datasets creating filenames
    num_labels = len(all_directories)
    num_classes_per_batch = params.num_classes_per_batch
    num_images_per_class = params.num_images_per_class
    
    
    def generator():
        while True:
            # Sample the labels that will compose the batch
            labels = np.random.choice(range(num_labels),
                                      num_classes_per_batch,
                                      replace=False)
            for label in labels:
                for _ in range(num_images_per_class):
                    yield label
    

    choice_dataset = tf.data.Dataset.from_generator(generator, tf.int64)
    dataset = tf.data.experimental.choose_from_datasets(datasets, choice_dataset)

    dataset = dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)

    batch_size = num_classes_per_batch * num_images_per_class

    dataset = dataset.batch(batch_size)
    dataset = dataset.repeat(params.num_epochs)
    dataset = dataset.prefetch(1)

    return dataset

In [ ]:
tf.reset_default_graph()
tf.logging.set_verbosity(tf.logging.INFO)

# Define the model
tf.logging.info("Creating the model...")
config = tf.estimator.RunConfig(tf_random_seed=230,
                                model_dir=model_dir,
                                save_summary_steps=params.save_summary_steps)
estimator = tf.estimator.Estimator(model_fn, params=params, config=config)

# Train the model
tf.logging.info("Starting training for {} epoch(s).".format(params.num_epochs))
estimator.train(lambda: train_input_fn(data_dir, params))

INFO:tensorflow:Creating the model...
INFO:tensorflow:Using config: {'_model_dir': 'experiments/batch_hard', '_tf_random_seed': 230, '_save_summary_steps': 50, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x15cb31080>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Starting training for 1000 epoch(s).
IN

INFO:tensorflow:loss = 1.6974581, step = 6001 (60.437 sec)
INFO:tensorflow:global_step/sec: 1.64034
INFO:tensorflow:loss = 1.6409782, step = 6101 (60.964 sec)
INFO:tensorflow:global_step/sec: 1.64616
INFO:tensorflow:loss = 1.6531155, step = 6201 (60.747 sec)
INFO:tensorflow:global_step/sec: 1.65515
INFO:tensorflow:loss = 1.6021565, step = 6301 (60.418 sec)
INFO:tensorflow:global_step/sec: 1.64948
INFO:tensorflow:loss = 1.6724757, step = 6401 (60.626 sec)
INFO:tensorflow:global_step/sec: 1.64745
INFO:tensorflow:loss = 1.657739, step = 6501 (60.700 sec)
INFO:tensorflow:global_step/sec: 1.64462
INFO:tensorflow:loss = 1.6058515, step = 6601 (60.804 sec)
INFO:tensorflow:global_step/sec: 1.60571
INFO:tensorflow:loss = 1.6337835, step = 6701 (62.282 sec)
INFO:tensorflow:Saving checkpoints for 6706 into experiments/batch_hard/model.ckpt.
INFO:tensorflow:global_step/sec: 1.55671
INFO:tensorflow:loss = 1.6896825, step = 6801 (64.234 sec)
INFO:tensorflow:global_step/sec: 1.64828
INFO:tensorflow:l

INFO:tensorflow:global_step/sec: 1.50398
INFO:tensorflow:loss = 1.6116551, step = 13601 (66.489 sec)
INFO:tensorflow:global_step/sec: 1.65826
INFO:tensorflow:loss = 1.6314555, step = 13701 (60.304 sec)
INFO:tensorflow:global_step/sec: 1.64171
INFO:tensorflow:loss = 1.6412759, step = 13801 (60.912 sec)
INFO:tensorflow:global_step/sec: 1.65172
INFO:tensorflow:loss = 1.6125509, step = 13901 (60.543 sec)
INFO:tensorflow:global_step/sec: 1.57051
INFO:tensorflow:loss = 1.6045816, step = 14001 (63.673 sec)
INFO:tensorflow:global_step/sec: 1.64855
INFO:tensorflow:loss = 1.5996076, step = 14101 (60.659 sec)
INFO:tensorflow:global_step/sec: 1.64451
INFO:tensorflow:loss = 1.6574837, step = 14201 (60.809 sec)
INFO:tensorflow:global_step/sec: 1.65288
INFO:tensorflow:loss = 1.6177577, step = 14301 (60.500 sec)
INFO:tensorflow:global_step/sec: 1.62865
INFO:tensorflow:loss = 1.6299376, step = 14401 (61.400 sec)
INFO:tensorflow:Saving checkpoints for 14476 into experiments/batch_hard/model.ckpt.
INFO:t

INFO:tensorflow:loss = 1.6341388, step = 21101 (60.974 sec)
INFO:tensorflow:global_step/sec: 1.65259
INFO:tensorflow:loss = 1.6172496, step = 21201 (60.510 sec)
INFO:tensorflow:Saving checkpoints for 21234 into experiments/batch_hard/model.ckpt.
INFO:tensorflow:global_step/sec: 1.53123
INFO:tensorflow:loss = 1.6146847, step = 21301 (65.311 sec)
INFO:tensorflow:global_step/sec: 1.65678
INFO:tensorflow:loss = 1.6451596, step = 21401 (60.354 sec)
INFO:tensorflow:global_step/sec: 1.64769
INFO:tensorflow:loss = 1.6464043, step = 21501 (60.691 sec)
INFO:tensorflow:global_step/sec: 1.64558
INFO:tensorflow:loss = 1.6309502, step = 21601 (60.769 sec)
INFO:tensorflow:global_step/sec: 1.66117
INFO:tensorflow:loss = 1.6228917, step = 21701 (60.199 sec)
INFO:tensorflow:global_step/sec: 1.63797
INFO:tensorflow:loss = 1.6145918, step = 21801 (61.051 sec)
INFO:tensorflow:global_step/sec: 1.64401
INFO:tensorflow:loss = 1.5264359, step = 21901 (60.832 sec)
INFO:tensorflow:global_step/sec: 1.64437
INFO:t

INFO:tensorflow:global_step/sec: 1.596
INFO:tensorflow:loss = 1.5676221, step = 28601 (62.673 sec)
INFO:tensorflow:global_step/sec: 1.59532
INFO:tensorflow:loss = 1.6199844, step = 28701 (62.668 sec)
INFO:tensorflow:global_step/sec: 1.55154
INFO:tensorflow:loss = 1.6089798, step = 28801 (64.452 sec)
INFO:tensorflow:global_step/sec: 1.43181
INFO:tensorflow:loss = 1.589106, step = 28901 (69.842 sec)
INFO:tensorflow:Saving checkpoints for 28983 into experiments/batch_hard/model.ckpt.
INFO:tensorflow:global_step/sec: 0.963994
INFO:tensorflow:loss = 1.637747, step = 29001 (103.735 sec)
INFO:tensorflow:global_step/sec: 1.6473
INFO:tensorflow:loss = 1.5496348, step = 29101 (60.705 sec)
INFO:tensorflow:global_step/sec: 1.64769
INFO:tensorflow:loss = 1.6052914, step = 29201 (60.691 sec)
INFO:tensorflow:global_step/sec: 1.64723
INFO:tensorflow:loss = 1.5700319, step = 29301 (60.708 sec)
INFO:tensorflow:global_step/sec: 1.65
INFO:tensorflow:loss = 1.6320885, step = 29401 (60.606 sec)
INFO:tensorf

...

INFO:tensorflow:global_step/sec: 1.64309
INFO:tensorflow:loss = 1.5990107, step = 58701 (60.860 sec)
INFO:tensorflow:global_step/sec: 1.62261
INFO:tensorflow:loss = 1.6157547, step = 58801 (61.630 sec)
INFO:tensorflow:global_step/sec: 1.64398
INFO:tensorflow:loss = 1.6325263, step = 58901 (60.832 sec)
INFO:tensorflow:global_step/sec: 1.64387
INFO:tensorflow:loss = 1.5598907, step = 59001 (60.832 sec)
INFO:tensorflow:Saving checkpoints for 59047 into experiments/batch_hard/model.ckpt.
INFO:tensorflow:global_step/sec: 1.53509
INFO:tensorflow:loss = 1.6220046, step = 59101 (65.143 sec)
INFO:tensorflow:global_step/sec: 1.62825
INFO:tensorflow:loss = 1.6443294, step = 59201 (61.411 sec)
INFO:tensorflow:global_step/sec: 1.63579
INFO:tensorflow:loss = 1.5942456, step = 59301 (61.133 sec)
INFO:tensorflow:global_step/sec: 1.63796
INFO:tensorflow:loss = 1.6114583, step = 59401 (61.052 sec)
INFO:tensorflow:global_step/sec: 1.62917
INFO:tensorflow:loss = 1.5980428, step = 59501 (61.381 sec)
INFO:t

INFO:tensorflow:global_step/sec: 1.48
INFO:tensorflow:loss = 1.578218, step = 66201 (67.569 sec)
INFO:tensorflow:global_step/sec: 1.58865
INFO:tensorflow:loss = 1.621928, step = 66301 (62.942 sec)
INFO:tensorflow:global_step/sec: 1.53753
INFO:tensorflow:loss = 1.603125, step = 66401 (65.040 sec)
INFO:tensorflow:global_step/sec: 1.57872
INFO:tensorflow:loss = 1.5836703, step = 66501 (63.346 sec)
INFO:tensorflow:global_step/sec: 1.55397
INFO:tensorflow:loss = 1.6140043, step = 66601 (64.347 sec)
INFO:tensorflow:global_step/sec: 1.59431
INFO:tensorflow:loss = 1.5921494, step = 66701 (62.726 sec)
INFO:tensorflow:global_step/sec: 1.6015
INFO:tensorflow:loss = 1.5963604, step = 66801 (62.438 sec)
INFO:tensorflow:Saving checkpoints for 66827 into experiments/batch_hard/model.ckpt.
INFO:tensorflow:global_step/sec: 1.49548
INFO:tensorflow:loss = 1.6015258, step = 66901 (66.868 sec)
INFO:tensorflow:global_step/sec: 1.61031
INFO:tensorflow:loss = 1.5581322, step = 67001 (62.099 sec)
INFO:tensorfl

INFO:tensorflow:global_step/sec: 1.62096
INFO:tensorflow:loss = 1.5660071, step = 73701 (61.695 sec)
INFO:tensorflow:global_step/sec: 1.61113
INFO:tensorflow:loss = 1.5146502, step = 73801 (62.071 sec)
INFO:tensorflow:global_step/sec: 1.64491
INFO:tensorflow:loss = 1.6075143, step = 73901 (60.788 sec)
INFO:tensorflow:global_step/sec: 1.6448
INFO:tensorflow:loss = 1.5994494, step = 74001 (60.798 sec)
INFO:tensorflow:global_step/sec: 1.63277
INFO:tensorflow:loss = 1.6076726, step = 74101 (61.246 sec)
INFO:tensorflow:global_step/sec: 1.62781
INFO:tensorflow:loss = 1.6105291, step = 74201 (61.432 sec)
INFO:tensorflow:global_step/sec: 1.6436
INFO:tensorflow:loss = 1.5772654, step = 74301 (60.842 sec)
INFO:tensorflow:global_step/sec: 1.63656
INFO:tensorflow:loss = 1.6076545, step = 74401 (61.104 sec)
INFO:tensorflow:global_step/sec: 1.65184
INFO:tensorflow:loss = 1.6130271, step = 74501 (60.538 sec)
INFO:tensorflow:Saving checkpoints for 74558 into experiments/batch_hard/model.ckpt.
INFO:ten

INFO:tensorflow:loss = 1.5885133, step = 81201 (60.204 sec)
INFO:tensorflow:Saving checkpoints for 81283 into experiments/batch_hard/model.ckpt.
INFO:tensorflow:global_step/sec: 1.53266
INFO:tensorflow:loss = 1.5488602, step = 81301 (65.242 sec)
INFO:tensorflow:global_step/sec: 1.65985
INFO:tensorflow:loss = 1.6156181, step = 81401 (60.247 sec)
INFO:tensorflow:global_step/sec: 1.64155
INFO:tensorflow:loss = 1.5922774, step = 81501 (60.918 sec)
INFO:tensorflow:global_step/sec: 1.6473
INFO:tensorflow:loss = 1.6028913, step = 81601 (60.706 sec)
INFO:tensorflow:global_step/sec: 1.63597
INFO:tensorflow:loss = 1.5791166, step = 81701 (61.126 sec)
INFO:tensorflow:global_step/sec: 1.62184
INFO:tensorflow:loss = 1.5946198, step = 81801 (61.659 sec)
INFO:tensorflow:global_step/sec: 1.64389
INFO:tensorflow:loss = 1.5941035, step = 81901 (60.831 sec)
INFO:tensorflow:global_step/sec: 1.64223
INFO:tensorflow:loss = 1.558537, step = 82001 (60.893 sec)
INFO:tensorflow:global_step/sec: 1.63267
INFO:ten

INFO:tensorflow:global_step/sec: 1.54459
INFO:tensorflow:loss = 1.6399046, step = 88701 (64.739 sec)
INFO:tensorflow:global_step/sec: 1.57746
INFO:tensorflow:loss = 1.5745442, step = 88801 (63.392 sec)
INFO:tensorflow:Saving checkpoints for 88876 into experiments/batch_hard/model.ckpt.
INFO:tensorflow:global_step/sec: 1.4087
INFO:tensorflow:loss = 1.5910548, step = 88901 (70.989 sec)
INFO:tensorflow:global_step/sec: 1.62851
INFO:tensorflow:loss = 1.6061682, step = 89001 (61.405 sec)
INFO:tensorflow:global_step/sec: 1.49302
INFO:tensorflow:loss = 1.600563, step = 89101 (66.991 sec)
INFO:tensorflow:global_step/sec: 1.55698
INFO:tensorflow:loss = 1.5948832, step = 89201 (64.215 sec)
INFO:tensorflow:global_step/sec: 1.63526
INFO:tensorflow:loss = 1.620032, step = 89301 (61.156 sec)
INFO:tensorflow:global_step/sec: 1.46025
INFO:tensorflow:loss = 1.5855403, step = 89401 (68.477 sec)
INFO:tensorflow:global_step/sec: 1.57226
INFO:tensorflow:loss = 1.5921735, step = 89501 (63.603 sec)
INFO:tens

# dimension test

In [110]:
tf.reset_default_graph()

In [111]:
num_channels = 32

In [112]:
image = tf.read_file('img/Marled_Knit_Racerback_Tank/img_00000002.jpg')

In [113]:
def preprocess_image(image):
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.convert_image_dtype(image, tf.float32)
        image = tf.math.divide(image, 255.0)     
        return tf.image.resize(image, [IMG_WIDTH, IMG_HEIGHT])

In [114]:
image = preprocess_image(image)

In [115]:
image = tf.reshape(image, (1, 150, 150, 3))

In [116]:
out = image
channels = [num_channels, num_channels * 2, num_channels, num_channels * 2]
for i, c in enumerate(channels):
    with tf.variable_scope('block_{}'.format(i+1)):
        out = tf.layers.conv2d(out, c, 3, padding='same')
        if params.use_batch_norm:
            out = tf.layers.batch_normalization(out, momentum=bn_momentum, training=is_training)
        out = tf.nn.relu(out)
        out = tf.layers.max_pooling2d(out, 2, 2)

In [117]:
out

<tf.Tensor 'block_4/max_pooling2d/MaxPool:0' shape=(1, 9, 9, 64) dtype=float32>

In [118]:
out.shape[1:] == [9, 9, num_channels * 2]

True

In [24]:
# Evaluate the model on the test set
tf.logging.info("Evaluation on test set.")
res = estimator.evaluate(lambda: test_input_fn(data_dir, params))
for key in res:
    print("{}: {}".format(key, res[key]))

INFO:tensorflow:Creating the model...
INFO:tensorflow:Using config: {'_model_dir': 'experiments/batch_hard', '_tf_random_seed': 230, '_save_summary_steps': 50, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x13627c160>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Starting training for 20 epoch(s).
Inst

INFO:tensorflow:loss = 0.100964844, step = 2701 (3.944 sec)
INFO:tensorflow:global_step/sec: 25.4464
INFO:tensorflow:loss = 0.40083218, step = 2801 (3.929 sec)
INFO:tensorflow:global_step/sec: 25.1949
INFO:tensorflow:loss = 0.14157505, step = 2901 (3.969 sec)
INFO:tensorflow:global_step/sec: 25.0165
INFO:tensorflow:loss = 0.1892492, step = 3001 (3.997 sec)
INFO:tensorflow:global_step/sec: 24.6855
INFO:tensorflow:loss = 0.042284787, step = 3101 (4.051 sec)
INFO:tensorflow:global_step/sec: 25.3373
INFO:tensorflow:loss = 0.108737186, step = 3201 (3.947 sec)
INFO:tensorflow:global_step/sec: 24.6682
INFO:tensorflow:loss = 0.1435358, step = 3301 (4.054 sec)
INFO:tensorflow:global_step/sec: 27.0328
INFO:tensorflow:loss = 0.12021956, step = 3401 (3.700 sec)
INFO:tensorflow:global_step/sec: 24.5658
INFO:tensorflow:loss = 0.05362767, step = 3501 (4.071 sec)
INFO:tensorflow:global_step/sec: 22.8474
INFO:tensorflow:loss = 0.096015036, step = 3601 (4.377 sec)
INFO:tensorflow:global_step/sec: 25.611

KeyboardInterrupt: 